In [1]:
from bert import my_bert_from_hf_weights
import transformers
import gin
import torchtext
import einops
import torch as t

tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
data_train, _ = torchtext.datasets.IMDB(root=".data", split=("train", "test"))
data_train = list(data_train)

100%|██████████| 84.1M/84.1M [00:02<00:00, 29.5MB/s]


In [3]:
labels = [x[0] for x in data_train]
text = [x[1] for x in data_train]

In [4]:
text[0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [42]:
def tokenize_text(t, max_seq_len = 512):
    return tokenizer(text[0], truncation=True, padding='max_length', max_length=max_seq_len)["input_ids"]
    
def preprocess_data(data, batch_size=8, max_seq_len=512):
    labels = [x[0] for x in data]
    text = [x[1] for x in data]
    final_inputs = []
    final_labels = []
    for i in range(len(text)):
        if len(text[i].strip().split(" ")) < 10:
            continue
        else:
            tokenized_text = tokenize_text(text[i], max_seq_len)
            final_inputs.append(tokenized_text)
            final_labels.append(0 if labels[i] == "neg" else 1)
            
    s = len(final_inputs) % batch_size
    final_inputs = einops.rearrange(t.tensor(final_inputs[s:]), "(b k) w -> b k w", k = batch_size)
    final_labels = einops.rearrange(t.tensor(final_labels[s:]), "(b k) -> b k", k = batch_size)
    return final_inputs, final_labels


In [43]:
processed_inputs, processed_labels = preprocess_data(data_train[0:10])

In [48]:
processed_inputs[-1].shape

torch.Size([8, 512])

In [47]:
processed_labels[0].shape

torch.Size([8])